# Apply color grading, enhance contrast and add scale bars to epifluorescence images

In [ ]:
# Tools to read in the image files and filenames
import glob
import os
import re 

# Calculation and data frame tools
import numpy as np
import pandas as pd

# Image processing tools
import skimage
import skimage.io
import skimage.morphology

___

This code will create a new folder in this directory called X_LUT containing tif files of the images with adjusted contrast (using the skimage.equalize_adapthist function), LUT and scale bar.

Name files as follows: "X_magnification_channel.tif" (for example: "X_10x_RFP.tif")

Written by Laura Luebbert, 22nd of March 2020.

___

# Load in the data:

Load images into Python:

In [ ]:
# Define the directory containing daytime data
data_dir = ''

# Glob string for images (loads all .tif files)
im_glob = os.path.join(data_dir, '*.tif')

# Get list of files in directory
im_list = sorted(glob.glob(im_glob))

# Let's look at the first 10 entries
im_list[:10]

# Define the parameters:

In [ ]:
# Define the interpixel distance in µm.
interpixel_distance_10x = 
interpixel_distance_20x = 

# Define the desired scale bar size in µm.
scale_bar = 

# Define green, blue or red LUT for each channel
blue_channel = "DAPI"
green_channel = "GFP"
red_channel = "RFP"

___

# Take a look at the first image:

Load the first tif file using scikit-image to get some information about bit-size and color channels:

In [ ]:
# Read in the first tif file using skimage.
# im_list[#] defines which image is displayed.
im = skimage.io.imread(im_list[0])

im.shape
# Let's get information about the image.
print("The image is stored as a", type(im), "of", im.dtype, "bits.", "The image consists of", im.shape[0], "x", im.shape[1], "pixels.")

___

# Enhance contrast and apply LUT according to channel:

In [ ]:
max_ims = []
zeros = np.zeros((1030, 1300))

# Set median filter for despeckling
selem = skimage.morphology.square(3)

for i, file in enumerate(im_list):
    # Read in each tif file using skimage
    max_image = skimage.io.imread(im_list[i])
    
    # local contrast enhancement based on histograms computed over different tile regions of the image
    max_image = skimage.exposure.equalize_adapthist(max_image)
    
    # Despeckle image
#    max_image = skimage.filters.rank.median(image, selem)
    
    ## Add color   
    if (im_list[i].split("_")[-1].split(".")[0] == blue_channel):
        merged_im = []
        
        # Append red channel
        # Since we don't have a red channel here, I'll append an array of zeros (Skimage needs all three channels to save rgb images.)
        merged_im.append(zeros)

        # Append green channel
        merged_im.append(zeros)

        # Append blue channel
        merged_im.append(max_image)
               
        # Reorder dimensions of array to allow us to display it with the bebi103 package (the new dimensions will be width x height (?) x number of channels)
        merged_im = np.moveaxis(merged_im, 0, 2)
        merged_im = np.array(merged_im)

        max_ims.append(merged_im)
        
        
    if (im_list[i].split("_")[-1].split(".")[0] == green_channel):
        merged_im = []
        
        # Append red channel
        merged_im.append(zeros)

        # Append green channel
        merged_im.append(max_image)

        # Append blue channel
        merged_im.append(zeros)
               
        # Reorder dimensions of array to allow us to display it with the bebi103 package (the new dimensions will be width x height (?) x number of channels)
        merged_im = np.moveaxis(merged_im, 0, 2)
        merged_im = np.array(merged_im)

        max_ims.append(merged_im)
        
        
    if (im_list[i].split("_")[-1].split(".")[0] == red_channel):
        merged_im = []
        
        # Append red channel
        merged_im.append(max_image)

        # Append green channel
        merged_im.append(zeros)

        # Append blue channel
        merged_im.append(zeros)
        
        merged_im = np.array(merged_im)

        # Reorder dimensions of array to allow us to display it with the bebi103 package (the new dimensions will be width x height (?) x number of channels)
        merged_im = np.moveaxis(merged_im, 0, 2)
        merged_im = np.array(merged_im)

        max_ims.append(merged_im)

# Save the images as an 8-bit tif file:

In [ ]:
# Scale down images to 8 bits to save them as a jpeg using skimage.
max_ims_8 = []

for i, file in enumerate(max_ims):
    # Linearly scale down to 8-bit.
    image = (max_ims[i]/max_ims[i].max())*255

    # Change list to array and change type to 8-bit array.
    image = np.array(image)
    image = image.astype(np.uint8)

    max_ims_8.append(image)

# Change entire array "ims" to 8-bit array.    
max_ims_8 = np.array(max_ims_8)
max_ims_8 = max_ims_8.astype(np.uint8)

### Burn in scale bars:

In [ ]:
max_ims_8_with_scalebars = []

scalebar_10 = (1/interpixel_distance_10x * scale_bar)
scalebar_20 = (1/interpixel_distance_20x * scale_bar)

for i, file in enumerate(max_ims_8):
    if (im_list[i].split("_")[-2] == "10x"):
        # Make a copy of the image.
        image_with_scalebar = max_ims_8[i]

        # Burn the scale bar by changing pixel values (scalebar = 50 µm, white scalebar = 1000 (for black set to = 0)).
        image_with_scalebar[970:980, 1100:1100+int(scalebar_10)] = 1000

        # Append to new array.
        max_ims_8_with_scalebars.append(image_with_scalebar)
        
    if (im_list[i].split("_")[-2] == "20x"):
        # Make a copy of the image.
        image_with_scalebar = max_ims_8[i]

        # Burn the scale bar by changing pixel values (scalebar = 50 µm, white scalebar = 1000 (for black set to = 0)).
        image_with_scalebar[970:980, 1100:1100+int(scalebar_20)] = 1000

        # Append to new array.
        max_ims_8_with_scalebars.append(image_with_scalebar)

# Display one image with the scalebar.
skimage.io.imshow(max_ims_8_with_scalebars[0])

### Create folder to save images to:

In [ ]:
# Create folder "X_max_singlechannel".
path = ("{}/{}_LUT").format(im_list[0].split("/")[-3], im_list[0].split("/")[-3])

os.mkdir(path)

### Slice out image names:

In [ ]:
# Save the filename of the image in array as a first step to get the image title.
files = []

for filename in im_list:
    files.append(filename.split("/")[-1])

# Save image titles in array   
imnames = []  

for name in files:
    imnames.append(name.split(".")[0])

### Save images:

In [ ]:
# Save all images with the scale bar.
for i, image in enumerate(max_ims_8_with_scalebars):
    skimage.io.imsave(
        ("{}/{}_LUT/{}_LUT.tif").format(im_list[0].split("/")[-3], im_list[0].split("/")[-3], imnames[i]),
        max_ims_8_with_scalebars[i],
        plugin=None,
        check_contrast=False,
    )

___

# Computing environment

In [ ]:
%load_ext watermark
%watermark -v -p re,numpy,pandas,skimage,jupyterlab